In [1]:
import pymc3 as pm
import numpy as np
import pandas as pd
import arviz as az
from matplotlib import pyplot as plt

Some exercises are taken from "Statistical Rethinking by Richard McElreath"

1. Which kind of parameters cannot be sampled using Hamiltonian Monte Carlo? Why?

The discrete (i.e., categorical) parameters cannot be sampled using HMC since the proposal distribution in HMS suggests the next target value by computing the gradient (i.e., derivative) of the posterior distribution with respect to the parameters.

2. Find the code you have implemented last week for reproducing Figure 12.1. Perhaps tweak the parameters so that the scatter plot looks similar to the one by Nikolaj above (narrow). Now sve this data and use it as training data for training a model (we are faking a situation as if this was real data).  The trained model can be the same, just make the priors weak, and add observation data.  Experiment with Metropolis, Gibbs and NUTS samplers in PyMC3 to learn posterior distributions over the paramaters $\theta_1$ and $\theta_2$.  Can you confirm that in this scenario the Hamiltonian sampler has much higher convergence ? (for instance higer ESS). Is it translating to faster sampling at better quality on the wall clock?

In [2]:
with pm.Model() as model_positive_correlation:
    mode = 0.58
    θ1 = pm.Normal('θ1', mu=mode, sd=0.08)
    
    θ2 = pm.Normal('θ2', θ1, 0.01)  
    
    θ1_minus_θ2 = pm.Deterministic (
            "θ1_minus_θ2", 
            θ1 - θ2
        )   

In [3]:
with model_positive_correlation:
    trace_NUTS_positive_correlation = pm.sample (1000, chains=4, cores=4)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [θ2, θ1]
Sampling 4 chains, 0 divergences: 100%|██████████| 6000/6000 [00:07<00:00, 794.41draws/s]
The number of effective samples is smaller than 25% for some parameters.


In [4]:
pm.save_trace(trace_NUTS_positive_correlation, "trace_NUTS_positive_correlation");

In [5]:
NUTS_Po = pm.load_trace('trace_NUTS_positive_correlation', model=model_positive_correlation)

In [6]:
Data = NUTS_Po.get_values('θ1_minus_θ2')

with pm.Model() as model_weak_prior:
    mode = 0.58
    θ1 = pm.Normal('θ1', mu=mode, sd=12) 
    
    θ2 = pm.Normal('θ2', mu=θ1, sd=0.1)
    
    θ3 = pm.Normal('θ3', mu=θ2, sd=0.1, observed=Data)  

In [7]:
with model_weak_prior:
    trace_weak_prior_NUTS = pm.sample (1000, cores=4)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [θ2, θ1]
Sampling 4 chains, 0 divergences: 100%|██████████| 6000/6000 [00:04<00:00, 1238.33draws/s]
The acceptance probability does not match the target. It is 0.8927917164185164, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8905414050514019, but should be close to 0.8. Try to increase the number of tuning steps.


In [8]:
pm.summary(trace_weak_prior_NUTS)

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
θ1,-0.0,0.102,-0.182,0.200,0.002,0.002,3878.0,1761.0,3874.0,2658.0,1.0
θ2,-0.0,0.002,-0.003,0.003,0.000,0.000,3665.0,1761.0,3662.0,2557.0,1.0


In [9]:
with model_weak_prior:
    trace_weak_prior_Metropolis = pm.sample (1000, cores=4, step=pm.Metropolis())

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [θ2]
>Metropolis: [θ1]
Sampling 4 chains, 0 divergences: 100%|██████████| 6000/6000 [00:01<00:00, 3561.78draws/s]
The estimated number of effective samples is smaller than 200 for some parameters.


In [10]:
pm.summary(trace_weak_prior_Metropolis)

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
θ1,-0.009,0.096,-0.211,0.152,0.004,0.004,551.0,355.0,569.0,565.0,1.01
θ2,0.000,0.002,-0.003,0.003,0.000,0.000,177.0,118.0,179.0,202.0,1.02


In [11]:
with model_weak_prior:
    trace_weak_prior_HamiltonianMC = pm.sample (1000, cores=4, step=pm.HamiltonianMC())

Multiprocess sampling (4 chains in 4 jobs)
HamiltonianMC: [θ2, θ1]
Sampling 4 chains, 0 divergences: 100%|██████████| 6000/6000 [00:23<00:00, 260.44draws/s] 
The acceptance probability does not match the target. It is 0.9945209080534148, but should be close to 0.65. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.7751039289577047, but should be close to 0.65. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8342640738570833, but should be close to 0.65. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9927707790135261, but should be close to 0.65. Try to increase the number of tuning steps.


In [12]:
pm.summary(trace_weak_prior_HamiltonianMC)

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
θ1,-0.002,0.103,-0.208,0.181,0.002,0.001,3219.0,2903.0,3223.0,2935.0,1.0
θ2,-0.000,0.002,-0.003,0.003,0.000,0.000,2230.0,2230.0,2213.0,2679.0,1.0
